In [1]:
from __future__ import print_function, division
from collections import Counter
import csv
import numpy as np
import random
import sys
import os
import copy
import time
from datetime import datetime,timedelta
from math import log, sqrt
import pandas as pd
import distance
from keras.models import load_model
import matplotlib.pyplot as plt
from jellyfish._jellyfish import damerau_levenshtein_distance

# 1.Read file

In [2]:
eventlog = "env_permit.csv"
eventlog_name = "env_permit"
ori_eventlog="env_permit.csv"

# 1.1. Calculate Maxlen and Target labels

to make sure the model is compile with different sized test set, the shape of the training set needs to be the same.
Thus, maxlen is loaded as the maximum length of a trace from the original log, target_chars is loaded as all possible characters from the original log

In [3]:
ori_csvfile = open('../data/%s' % ori_eventlog, 'r')
print('original dataset',ori_eventlog)
spamreader = csv.reader(ori_csvfile, delimiter=',', quotechar='|')
next(spamreader, None)  # skip the headers
ascii_offset = 161
lines = [] #these are all the activity seq
timeseqs = [] #time sequences (differences between two events)
timeseqs2 = [] #time sequences (differences between the current and first)

#helper variables
lastcase = ''
line = ''
firstLine = True
times = []
times2 = []
numlines = 0
casestarttime = None
lasteventtime = None
for row in spamreader: #the rows are "CaseID,ActivityID,CompleteTimestamp"
    t = datetime.strptime(row[2], "%Y-%m-%d %H:%M:%S") #creates a datetime object from row[2]
    if row[0]!=lastcase:  #'lastcase' is to save the last executed case for the loop
        casestarttime = t
        lasteventtime = t
        lastcase = row[0]
        if not firstLine:
            lines.append(line)
            timeseqs.append(times)
            timeseqs2.append(times2)
        line = ''
        times = []
        times2 = []
        numlines+=1
    line+=chr(int(row[1])+ascii_offset)
    timesincelastevent = t - lasteventtime
    timesincecasestart = t - casestarttime
    timediff = 86400 * timesincelastevent.days + timesincelastevent.seconds  #time b/t current and last event
    timediff2 = 86400 * timesincecasestart.days + timesincecasestart.seconds #time b/t current and starting event
    times.append(timediff)
    times2.append(timediff2)
    lasteventtime = t
    firstLine = False
# add last case
lines.append(line)
timeseqs.append(times)
timeseqs2.append(times2)
numlines+=1
step = 1
sentences = []
softness = 0
next_chars = []
lines = list(map(lambda x: x+ '!',lines)) #add a delimiter symbol '!' to the end of each line
maxlen = max(map(lambda x: len(x),lines)) #find maximum line size
print('maxlen:',maxlen)
# next lines here to get all possible characters for events and annotate them with numbers
chars = list(map(lambda x: set(x),lines))
chars = list(set().union(*chars))
chars.sort()
target_chars = copy.copy(chars)
if '!' in chars:
    chars.remove('!')
print('total chars: {}, target chars: {}'.format(len(chars), len(target_chars)))
#get the target chars from the training set
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
target_char_indices = dict((c, i) for i, c in enumerate(target_chars))
target_indices_char = dict((i, c) for i, c in enumerate(target_chars))
print(indices_char)

original dataset env_permit.csv
maxlen: 96
total chars: 381, target chars: 382
{0: '¢', 1: '£', 2: '¤', 3: '¥', 4: '¦', 5: '§', 6: '¨', 7: '©', 8: 'ª', 9: '«', 10: '¬', 11: '\xad', 12: '®', 13: '¯', 14: '°', 15: '±', 16: '²', 17: '³', 18: '´', 19: 'µ', 20: '¶', 21: '·', 22: '¸', 23: '¹', 24: 'º', 25: '»', 26: '¼', 27: '½', 28: '¾', 29: '¿', 30: 'À', 31: 'Á', 32: 'Â', 33: 'Ã', 34: 'Ä', 35: 'Å', 36: 'Æ', 37: 'Ç', 38: 'È', 39: 'É', 40: 'Ê', 41: 'Ë', 42: 'Ì', 43: 'Í', 44: 'Î', 45: 'Ï', 46: 'Ð', 47: 'Ñ', 48: 'Ò', 49: 'Ó', 50: 'Ô', 51: 'Õ', 52: 'Ö', 53: '×', 54: 'Ø', 55: 'Ù', 56: 'Ú', 57: 'Û', 58: 'Ü', 59: 'Ý', 60: 'Þ', 61: 'ß', 62: 'à', 63: 'á', 64: 'â', 65: 'ã', 66: 'ä', 67: 'å', 68: 'æ', 69: 'ç', 70: 'è', 71: 'é', 72: 'ê', 73: 'ë', 74: 'ì', 75: 'í', 76: 'î', 77: 'ï', 78: 'ð', 79: 'ñ', 80: 'ò', 81: 'ó', 82: 'ô', 83: 'õ', 84: 'ö', 85: '÷', 86: 'ø', 87: 'ù', 88: 'ú', 89: 'û', 90: 'ü', 91: 'ý', 92: 'þ', 93: 'ÿ', 94: 'Ā', 95: 'ā', 96: 'Ă', 97: 'ă', 98: 'Ą', 99: 'ą', 100: 'Ć', 101: 'ć', 102: 'Ĉ

# 2.Preprocessing

In [4]:
csvfile = open('../data/%s' % eventlog, 'r')
spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
next(spamreader, None)  # skip the headers
ascii_offset = 161
lines = [] #these are all the activity seq
timeseqs = [] #time sequences (differences between two events)
timeseqs2 = [] #time sequences (differences between the current and first)

#helper variables
lastcase = ''
line = ''
firstLine = True
times = []
times2 = []
numlines = 0
casestarttime = None
lasteventtime = None

In [5]:
for row in spamreader: #the rows are "CaseID,ActivityID,CompleteTimestamp"
    t = datetime.strptime(row[2], "%Y-%m-%d %H:%M:%S") #creates a datetime object from row[2]
    if row[0]!=lastcase:  #'lastcase' is to save the last executed case for the loop
        casestarttime = t
        lasteventtime = t
        lastcase = row[0]
        if not firstLine:
            lines.append(line)
            timeseqs.append(times)
            timeseqs2.append(times2)
        line = ''
        times = []
        times2 = []
        numlines+=1
    line+=chr(int(row[1])+ascii_offset)
    timesincelastevent = t - lasteventtime
    timesincecasestart = t - casestarttime
    timediff = 86400 * timesincelastevent.days + timesincelastevent.seconds  #time b/t current and last event
    timediff2 = 86400 * timesincecasestart.days + timesincecasestart.seconds #time b/t current and starting event
    times.append(timediff)
    times2.append(timediff2)
    lasteventtime = t
    firstLine = False

# add last case
lines.append(line)
timeseqs.append(times)
timeseqs2.append(times2)
numlines+=1

In [6]:
# Example of accessing processed data
#print(lines)
#print(timeseqs)
#print(timeseqs2)

In [7]:
#average time between events
divisor = np.mean([item for sublist in timeseqs for item in sublist]) 
print('divisor: {}'.format(divisor))
#average time between current and starting events
divisor2 = np.mean([item for sublist in timeseqs2 for item in sublist]) 
print('divisor2: {}'.format(divisor2))


divisor: 186908.12533381264
divisor2: 5159390.008756163


# 3. Feature Enginnering

In [8]:
csvfile = open('../data/%s' % eventlog, 'r')
spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
next(spamreader, None)  # skip the headers
lastcase = ''
line = ''
firstLine = True
lines = []
timeseqs = []
timeseqs2 = []
timeseqs3 = []
timeseqs4 = []
times = []
times2 = []
times3 = []
times4 = []
numlines = 0
casestarttime = None
lasteventtime = None

In [9]:
for row in spamreader:
    t = time.strptime(row[2], "%Y-%m-%d %H:%M:%S")
    if row[0]!=lastcase:
        casestarttime = t
        lasteventtime = t
        lastcase = row[0]
        if not firstLine:
            lines.append(line)
            timeseqs.append(times)
            timeseqs2.append(times2)
            timeseqs3.append(times3)
            timeseqs4.append(times4)
        line = ''
        times = []
        times2 = []
        times3 = []
        times4 = []
        numlines+=1
    line+=chr(int(row[1])+ascii_offset)
    timesincelastevent = datetime.fromtimestamp(time.mktime(t))-datetime.fromtimestamp(time.mktime(lasteventtime))
    timesincecasestart = datetime.fromtimestamp(time.mktime(t))-datetime.fromtimestamp(time.mktime(casestarttime))
    midnight = datetime.fromtimestamp(time.mktime(t)).replace(hour=0, minute=0, second=0, microsecond=0)
    timesincemidnight = datetime.fromtimestamp(time.mktime(t))-midnight
    timediff = 86400 * timesincelastevent.days + timesincelastevent.seconds
    timediff2 = 86400 * timesincecasestart.days + timesincecasestart.seconds
    timediff3 = timesincemidnight.seconds #this leaves only time even occurred after midnight
    timediff4 = datetime.fromtimestamp(time.mktime(t)).weekday() #day of the week
    times.append(timediff)
    times2.append(timediff2)
    times3.append(timediff3)
    times4.append(timediff4)
    lasteventtime = t
    firstLine = False

# add last case
lines.append(line)
timeseqs.append(times)
timeseqs2.append(times2)
timeseqs3.append(times3)
timeseqs4.append(times4)
numlines+=1

In [10]:
# fold 1
elems_per_fold = int(round(numlines/3)) #calculate the number of elements per fold
fold1 = lines[:elems_per_fold]
fold1_t = timeseqs[:elems_per_fold]
fold1_t2 = timeseqs2[:elems_per_fold]
fold1_t3 = timeseqs3[:elems_per_fold]
fold1_t4 = timeseqs4[:elems_per_fold]
  
# fold 2
fold2 = lines[elems_per_fold:2*elems_per_fold]
fold2_t = timeseqs[elems_per_fold:2*elems_per_fold]
fold2_t2 = timeseqs2[elems_per_fold:2*elems_per_fold]
fold2_t3 = timeseqs3[elems_per_fold:2*elems_per_fold]
fold2_t4 = timeseqs4[elems_per_fold:2*elems_per_fold]
        
# fold 3
fold3 = lines[2*elems_per_fold:]
fold3_t = timeseqs[2*elems_per_fold:]
fold3_t2 = timeseqs2[2*elems_per_fold:]
fold3_t3 = timeseqs3[2*elems_per_fold:]
fold3_t4 = timeseqs4[2*elems_per_fold:]

lines = fold1 + fold2
lines_t = fold1_t + fold2_t
lines_t2 = fold1_t2 + fold2_t2
lines_t3 = fold1_t3 + fold2_t3
lines_t4 = fold1_t4 + fold2_t4

In [11]:
#save training set
df_train = pd.DataFrame({
    'lines': lines,
    'times': lines_t,
    'times2': lines_t2,
    'times3': lines_t3,
    'times4': lines_t4
    })
# Convert times columns to strings
df_train['times'] = df_train['times'].apply(lambda x: ','.join(map(str, x)))
df_train['times2'] = df_train['times2'].apply(lambda x: ','.join(map(str, x)))
df_train['times3'] = df_train['times3'].apply(lambda x: ','.join(map(str, x)))
df_train['times4'] = df_train['times4'].apply(lambda x: ','.join(map(str, x)))
# Decode the 'lines' column from ASCII to integers
df_train['lines'] = df_train['lines'].apply(lambda x: ''.join(str(ord(c) - ascii_offset) for c in x))
df_train.to_excel(f'output_files/folds/{eventlog_name}_train_set.xlsx', engine='xlsxwriter', index=False)

#save test set
df_test = pd.DataFrame({
    'lines': fold3,
    'times': fold3_t,
    'times2': fold3_t2,
    'times3': fold3_t3,
    'times4': fold3_t4
    })
# Convert times columns to strings
df_test['times'] = df_test['times'].apply(lambda x: ','.join(map(str, x)))
df_test['times2'] = df_test['times2'].apply(lambda x: ','.join(map(str, x)))
df_test['times3'] = df_test['times3'].apply(lambda x: ','.join(map(str, x)))
df_test['times4'] = df_test['times4'].apply(lambda x: ','.join(map(str, x)))
# Decode the 'lines' column from ASCII to integers
df_test['lines'] = df_test['lines'].apply(lambda x: ''.join(str(ord(c) - ascii_offset) for c in x))
df_test.to_excel(f'output_files/folds/{eventlog_name}_test_set.xlsx', engine='xlsxwriter', index=False)

In [12]:
step = 1
sentences = []
softness = 0
next_chars = []
lines = list(map(lambda x: x+'!', lines))

sentences_t = []
sentences_t2 = []
sentences_t3 = []
sentences_t4 = []
next_chars_t = []
next_chars_t2 = []
next_chars_t3 = []
next_chars_t4 = []

for line, line_t, line_t2, line_t3, line_t4 in zip(lines, lines_t, lines_t2, lines_t3, lines_t4):
    for i in range(0, len(line), step):
        if i==0:
            continue

        #we add iteratively, first symbol of the line, then two first, three...
        sentences.append(line[0: i])
        sentences_t.append(line_t[0:i])
        sentences_t2.append(line_t2[0:i])
        sentences_t3.append(line_t3[0:i])
        sentences_t4.append(line_t4[0:i])
        next_chars.append(line[i])

        if i==len(line)-1: # special case to deal time of end character
            next_chars_t.append(0)
            next_chars_t2.append(0)
            next_chars_t3.append(0)
            next_chars_t4.append(0)
        else:
            next_chars_t.append(line_t[i])
            next_chars_t2.append(line_t2[i])
            next_chars_t3.append(line_t3[i])
            next_chars_t4.append(line_t4[i])

print('nb sequences:', len(sentences))

nb sequences: 24600


# 4. Encoding

In [13]:
print('Vectorization...')
num_features = len(chars)+5
print('num features: {}'.format(num_features))
X = np.zeros((len(sentences), maxlen, num_features), dtype=np.float32)
y_a = np.zeros((len(sentences), len(target_chars)), dtype=np.float32)
y_t = np.zeros((len(sentences)), dtype=np.float32)
for i, sentence in enumerate(sentences):
    leftpad = maxlen-len(sentence)
    next_t = next_chars_t[i]
    sentence_t = sentences_t[i]
    sentence_t2 = sentences_t2[i]
    sentence_t3 = sentences_t3[i]
    sentence_t4 = sentences_t4[i]
    for t, char in enumerate(sentence):
        multiset_abstraction = Counter(sentence[:t+1])
        for c in chars:
            if c==char: #this will encode present events to the right places
                X[i, t+leftpad, char_indices[c]] = 1
        X[i, t+leftpad, len(chars)] = t+1
        X[i, t+leftpad, len(chars)+1] = sentence_t[t]/divisor
        X[i, t+leftpad, len(chars)+2] = sentence_t2[t]/divisor2
        X[i, t+leftpad, len(chars)+3] = sentence_t3[t]/86400
        X[i, t+leftpad, len(chars)+4] = sentence_t4[t]/7
    for c in target_chars:
        if c==next_chars[i]:
            y_a[i, target_char_indices[c]] = 1-softness
        else:
            y_a[i, target_char_indices[c]] = softness/(len(target_chars)-1)
    y_t[i] = next_t/divisor
    np.set_printoptions(threshold=sys.maxsize)

Vectorization...
num features: 386


# 5. Train DNN

In [14]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, BatchNormalization,Flatten
from tensorflow.keras.optimizers import Nadam, legacy
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [15]:
print('Build model...')
main_input = Input(shape=(maxlen, num_features), name='main_input')
main_input

Build model...


<KerasTensor: shape=(None, 96, 386) dtype=float32 (created by layer 'main_input')>

In [16]:
flatten_layer = Flatten()(main_input)
dense1 = Dense(100, activation='relu', kernel_initializer='glorot_uniform')(flatten_layer)
dense1 = BatchNormalization()(dense1)
dense2 = Dense(100, activation='relu', kernel_initializer='glorot_uniform')(dense1)
dense2 = BatchNormalization()(dense2)
act_output = Dense(len(target_chars), activation='softmax', kernel_initializer='glorot_uniform', name='act_output')(dense2)
#time_output = Dense(1, kernel_initializer='glorot_uniform', name='time_output')(dense2)

In [17]:
#model = Model(inputs=[main_input], outputs=[act_output, time_output])
model = Model(inputs=[main_input], outputs=[act_output])

In [18]:
checkpoint_filepath = os.path.join('output_files/models', f'{eventlog_name}_DNN_model_{{epoch:02d}}-{{val_loss:.2f}}.h5')

In [19]:
opt = legacy.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.004, clipvalue=3)
#model.compile(loss={'act_output': 'categorical_crossentropy', 'time_output': 'mae'}, optimizer=opt)
model.compile(loss={'act_output': 'categorical_crossentropy'}, optimizer=opt)
early_stopping = EarlyStopping(monitor='val_loss', patience=42)
model_checkpoint = ModelCheckpoint(filepath=checkpoint_filepath, monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto')
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)

C:\Users\I541451\Anaconda3\lib\site-packages\keras\optimizers\legacy\nadam.py:89: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [20]:
#model.fit(X, {'act_output':y_a, 'time_output':y_t}, validation_split=0.2, verbose=2, callbacks=[early_stopping, model_checkpoint, lr_reducer], batch_size=maxlen, epochs=500)
model.fit(X, {'act_output':y_a}, validation_split=0.2, verbose=2, callbacks=[early_stopping, model_checkpoint, lr_reducer], batch_size=maxlen, epochs=500)

Epoch 1/500
205/205 - 10s - loss: 3.0768 - val_loss: 4.5316 - lr: 0.0020 - 10s/epoch - 49ms/step
Epoch 2/500
205/205 - 8s - loss: 1.9481 - val_loss: 4.0104 - lr: 0.0020 - 8s/epoch - 40ms/step
Epoch 3/500
205/205 - 8s - loss: 1.5747 - val_loss: 3.9915 - lr: 0.0020 - 8s/epoch - 37ms/step
Epoch 4/500
205/205 - 8s - loss: 1.3210 - val_loss: 3.8148 - lr: 0.0020 - 8s/epoch - 37ms/step
Epoch 5/500
205/205 - 8s - loss: 1.1533 - val_loss: 4.0430 - lr: 0.0020 - 8s/epoch - 37ms/step
Epoch 6/500
205/205 - 22s - loss: 1.0298 - val_loss: 3.9926 - lr: 0.0020 - 22s/epoch - 106ms/step
Epoch 7/500
205/205 - 7s - loss: 0.9173 - val_loss: 4.6048 - lr: 0.0020 - 7s/epoch - 36ms/step
Epoch 8/500
205/205 - 8s - loss: 0.8224 - val_loss: 4.2708 - lr: 0.0020 - 8s/epoch - 40ms/step
Epoch 9/500
205/205 - 10s - loss: 0.7606 - val_loss: 4.4768 - lr: 0.0020 - 10s/epoch - 48ms/step
Epoch 10/500
205/205 - 9s - loss: 0.6783 - val_loss: 4.5940 - lr: 0.0020 - 9s/epoch - 43ms/step
Epoch 11/500
205/205 - 9s - loss: 0.6427 -